In [131]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, DenseNet121
from tensorflow.keras.layers import Input,Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix

In [132]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, GlobalAveragePooling2D, Dense

In [133]:


# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Preprocess the data
train_images = train_images / 255.0
test_images = test_images / 255.0
train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)

# Data augmentation for the training set
datagen = ImageDataGenerator(
    rotation_range=20,          # Randomly rotate images in the range (-20, 20) degrees
    width_shift_range=0.1,      # Randomly shift images horizontally by up to 10% of the width
    height_shift_range=0.1,     # Randomly shift images vertically by up to 10% of the height
    horizontal_flip=True,       # Randomly flip images horizontally
    zoom_range=0.1,             # Randomly zoom in/out by up to 10%
    fill_mode='nearest',        # Filling mode for new pixels created after rotation or shifting
    cval=0,                     # Value used for points outside the boundaries with constant fill mode
    rescale=None,               # Rescaling factor, if any (no need since we already scaled the data)
    preprocessing_function=None  # Optional custom preprocessing function
)

# Fit the augmenter to your data
datagen.fit(train_images)

# Generate augmented data
augmented_data = datagen.flow(train_images, train_labels, batch_size=64)

# Example: To get a batch of augmented data
batch_images, batch_labels = next(augmented_data)


In [134]:
# Define the learning rate schedule
def lr_schedule(epoch):
    if epoch < 200:
        return 0.1
    elif epoch < 400:
        return 0.01
    else:
        return 0.001


In [135]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, GlobalAveragePooling2D, Dense

def create_base_net(num_layers=20):
    input_layer = Input(shape=(32, 32, 3))
    
    x = Conv2D(16, (3, 3), padding='same', activation='relu')(input_layer)
    
    # Define the number of convolution layers per module
    num_layers_per_module = [6, 6, 6]  # You can adjust these as needed
    
    in_channels = 16
    
    for num_layers in num_layers_per_module:
        for _ in range(num_layers):
            x = Conv2D(in_channels, (3, 3), padding='same', activation='relu')(x)
            x = BatchNormalization()(x)
        
        in_channels *= 2  # Double the number of channels after each module
    
    x = GlobalAveragePooling2D()(x)
    output_layer = Dense(10, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    
    return model

# Usage:
model_base_20 = create_base_net(num_layers=20)


In [136]:
# Train and evaluate BaseNet-20
model_base_20 = create_base_net(num_layers=20)
model_base_20.compile(optimizer=SGD(learning_rate=0.1, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model_base_20.fit(train_images, train_labels, batch_size=64, epochs=10, callbacks=[LearningRateScheduler(lr_schedule)])
scores_base_20 = model_base_20.evaluate(train_images, train_labels, verbose=0)
print("BaseNet-20 Training Accuracy:", scores_base_20[1])
print("BaseNet-20 Training Loss:", scores_base_20[0])

Epoch 1/10
782/782 [==============================] - 27s 25ms/step - loss: 2.0916 - accuracy: 0.2069 - lr: 0.1000
Epoch 2/10
782/782 [==============================] - 20s 25ms/step - loss: 2.0396 - accuracy: 0.2207 - lr: 0.1000
Epoch 3/10
782/782 [==============================] - 20s 25ms/step - loss: 2.0172 - accuracy: 0.2300 - lr: 0.1000
Epoch 4/10
782/782 [==============================] - 20s 25ms/step - loss: 2.0136 - accuracy: 0.2311 - lr: 0.1000
Epoch 5/10
782/782 [==============================] - 20s 25ms/step - loss: 2.0158 - accuracy: 0.2302 - lr: 0.1000
Epoch 6/10
782/782 [==============================] - 20s 25ms/step - loss: 2.0124 - accuracy: 0.2305 - lr: 0.1000
Epoch 7/10
782/782 [==============================] - 20s 25ms/step - loss: 2.0176 - accuracy: 0.2283 - lr: 0.1000
Epoch 8/10
782/782 [==============================] - 19s 25ms/step - loss: 2.0261 - accuracy: 0.2267 - lr: 0.1000
Epoch 9/10
782/782 [==============================] - 20s 25ms/step - loss: 2.04

In [137]:
# Evaluate BaseNet-20 on the testing data with integer labels
test_loss_base_20, test_acc_base_20 = model_base_20.evaluate(test_images, test_labels, verbose=2)
print(f'BaseNet-20 Testing Accuracy: {test_acc_base_20:.4f}')
print(f'BaseNet-20 Testing Loss: {test_loss_base_20:.4f}')


313/313 - 1s - loss: 3.1207 - accuracy: 0.1200 - 1s/epoch - 5ms/step
BaseNet-20 Testing Accuracy: 0.1200
BaseNet-20 Testing Loss: 3.1207


In [138]:
# Train and evaluate BaseNet-32
model_base_32 = create_base_net(num_layers=32)
model_base_32.compile(optimizer=SGD(learning_rate=0.1, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model_base_32.fit(train_images, train_labels, batch_size=64, epochs=10, callbacks=[LearningRateScheduler(lr_schedule)])
scores_base_32 = model_base_32.evaluate(train_images, train_labels, verbose=0)
print("BaseNet-32 Training Accuracy:", scores_base_32[1])
print("BaseNet-32 Training Loss:", scores_base_32[0])

Epoch 1/10
782/782 [==============================] - 24s 25ms/step - loss: 2.0851 - accuracy: 0.2134 - lr: 0.1000
Epoch 2/10
782/782 [==============================] - 20s 25ms/step - loss: 2.0125 - accuracy: 0.2233 - lr: 0.1000
Epoch 3/10
782/782 [==============================] - 19s 25ms/step - loss: 1.9728 - accuracy: 0.2342 - lr: 0.1000
Epoch 4/10
782/782 [==============================] - 19s 25ms/step - loss: 1.9885 - accuracy: 0.2340 - lr: 0.1000
Epoch 5/10
782/782 [==============================] - 20s 25ms/step - loss: 1.9555 - accuracy: 0.2459 - lr: 0.1000
Epoch 6/10
782/782 [==============================] - 20s 25ms/step - loss: 1.9343 - accuracy: 0.2541 - lr: 0.1000
Epoch 7/10
782/782 [==============================] - 20s 25ms/step - loss: 2.0059 - accuracy: 0.2336 - lr: 0.1000
Epoch 8/10
782/782 [==============================] - 19s 25ms/step - loss: 1.9998 - accuracy: 0.2359 - lr: 0.1000
Epoch 9/10
782/782 [==============================] - 19s 25ms/step - loss: 2.00

In [139]:
# Evaluate BaseNet-20 on the testing data with integer labels
test_loss_base_32, test_acc_base_32 = model_base_32.evaluate(test_images, test_labels, verbose=2)
print(f'BaseNet-32 Testing Accuracy: {test_acc_base_32:.4f}')
print(f'BaseNet-32 Testing Loss: {test_loss_base_32:.4f}')


313/313 - 1s - loss: 2.2216 - accuracy: 0.2147 - 1s/epoch - 5ms/step
BaseNet-32 Testing Accuracy: 0.2147
BaseNet-32 Testing Loss: 2.2216


In [140]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, GlobalAveragePooling2D, Dense, Add

def residual_block(x, filters, stride=1):
    shortcut = x
    x = Conv2D(filters, (3, 3), strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    
    if shortcut.shape[1:] != x.shape[1:]:
        shortcut = Conv2D(filters, (1, 1), strides=stride, padding='valid')(shortcut)
    
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    
    return x

def create_resnet(num_layers=20):
    input_layer = Input(shape=(32, 32, 3))
    
    x = Conv2D(16, (3, 3), padding='same', activation='relu')(input_layer)
    
    num_modules = 3
    num_blocks_per_module = 3
    
    in_channels = 16
    
    for module in range(num_modules):
        for block in range(num_blocks_per_module):
            x = residual_block(x, in_channels)
        in_channels *= 2
    
    x = GlobalAveragePooling2D()(x)
    output_layer = Dense(10, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    
    return model

# Usage:
model_resnet_20 = create_resnet(num_layers=20)

In [141]:
# Train and evaluate ResNet-20
model_resnet_20 = create_resnet(num_layers=20)
model_resnet_20.compile(optimizer=SGD(learning_rate=0.1, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model_resnet_20.fit(train_images, train_labels, batch_size=64, epochs=10, callbacks=[LearningRateScheduler(lr_schedule)])
scores_resnet_20 = model_resnet_20.evaluate(train_images, train_labels, verbose=0)
print("ResNet-20 Training Accuracy:", scores_resnet_20[1])
print("ResNet-20 Training Loss:", scores_resnet_20[0])


Epoch 1/10
782/782 [==============================] - 27s 28ms/step - loss: 1.7288 - accuracy: 0.3516 - lr: 0.1000
Epoch 2/10
782/782 [==============================] - 22s 28ms/step - loss: 1.2668 - accuracy: 0.5390 - lr: 0.1000
Epoch 3/10
782/782 [==============================] - 22s 28ms/step - loss: 1.0307 - accuracy: 0.6289 - lr: 0.1000
Epoch 4/10
782/782 [==============================] - 22s 28ms/step - loss: 0.9080 - accuracy: 0.6769 - lr: 0.1000
Epoch 5/10
782/782 [==============================] - 22s 28ms/step - loss: 0.8190 - accuracy: 0.7118 - lr: 0.1000
Epoch 6/10
782/782 [==============================] - 22s 28ms/step - loss: 0.7480 - accuracy: 0.7362 - lr: 0.1000
Epoch 7/10
782/782 [==============================] - 22s 28ms/step - loss: 0.6938 - accuracy: 0.7573 - lr: 0.1000
Epoch 8/10
782/782 [==============================] - 22s 28ms/step - loss: 0.6351 - accuracy: 0.7768 - lr: 0.1000
Epoch 9/10
782/782 [==============================] - 22s 28ms/step - loss: 0.59

In [142]:
# Evaluate ResNet-20 on the testing data with integer labels
test_loss_resnet_20, test_acc_resnet_20 = model_resnet_20.evaluate(test_images, test_labels, verbose=2)
print(f'ResNet-20 Testing Accuracy: {test_acc_resnet_20:.4f}')
print(f'ResNet-20 Testing Loss: {test_loss_resnet_20:.4f}')


313/313 - 2s - loss: 1.0231 - accuracy: 0.6833 - 2s/epoch - 5ms/step
ResNet-20 Testing Accuracy: 0.6833
ResNet-20 Testing Loss: 1.0231


In [143]:
# Train and evaluate ResNet-32
model_resnet_32 = create_resnet(num_layers=32)
model_resnet_32.compile(optimizer=SGD(learning_rate=0.1, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model_resnet_32.fit(train_images, train_labels, batch_size=64, epochs=10, callbacks=[LearningRateScheduler(lr_schedule)])
scores_resnet_32 = model_resnet_32.evaluate(train_images, train_labels, verbose=0)
print("ResNet-32 Training Accuracy:", scores_resnet_32[1])
print("ResNet-32 Training Loss:", scores_resnet_32[0])

Epoch 1/10
782/782 [==============================] - 27s 28ms/step - loss: 1.6927 - accuracy: 0.3575 - lr: 0.1000
Epoch 2/10
782/782 [==============================] - 22s 28ms/step - loss: 1.2870 - accuracy: 0.5300 - lr: 0.1000
Epoch 3/10
782/782 [==============================] - 22s 28ms/step - loss: 1.0782 - accuracy: 0.6131 - lr: 0.1000
Epoch 4/10
782/782 [==============================] - 22s 28ms/step - loss: 0.9660 - accuracy: 0.6545 - lr: 0.1000
Epoch 5/10
782/782 [==============================] - 22s 28ms/step - loss: 0.8601 - accuracy: 0.6945 - lr: 0.1000
Epoch 6/10
782/782 [==============================] - 22s 28ms/step - loss: 0.7811 - accuracy: 0.7263 - lr: 0.1000
Epoch 7/10
782/782 [==============================] - 22s 28ms/step - loss: 0.7173 - accuracy: 0.7495 - lr: 0.1000
Epoch 8/10
782/782 [==============================] - 22s 28ms/step - loss: 0.6693 - accuracy: 0.7652 - lr: 0.1000
Epoch 9/10
782/782 [==============================] - 22s 28ms/step - loss: 0.62

In [144]:
# Evaluate ResNet-20 on the testing data with integer labels
test_loss_resnet_32, test_acc_resnet_32 = model_resnet_32.evaluate(test_images, test_labels, verbose=2)
print(f'ResNet-32 Testing Accuracy: {test_acc_resnet_32:.4f}')
print(f'ResNet-32 Testing Loss: {test_loss_resnet_32:.4f}')


313/313 - 2s - loss: 1.0830 - accuracy: 0.6669 - 2s/epoch - 5ms/step
ResNet-32 Testing Accuracy: 0.6669
ResNet-32 Testing Loss: 1.0830


**Using the Adam Optimizer**

In [145]:
from tensorflow.keras.optimizers import Adam  # Use the legacy optimizer

In [146]:
from tensorflow.keras.optimizers import Adam  # Use the legacy optimizer
# Initialize the legacy Adam optimizer
adam = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)


# Train and evaluate BaseNet-32 with the Adam optimizer
model_base_32_adam = create_base_net(num_layers=32)
model_base_32_adam.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model_base_32_adam.fit(train_images, train_labels, batch_size=64, epochs=10, callbacks=[LearningRateScheduler(lr_schedule)])
scores_base_32_adam = model_base_32_adam.evaluate(train_images, train_labels, verbose=0)
print("BaseNet-32 (Adam) Training Accuracy:", scores_base_32_adam[1])
print("BaseNet-32 (Adam) Training Loss:", scores_base_32_adam[0])

test_loss_base_32_adam, test_acc_base_32_adam = model_base_32_adam.evaluate(test_images, test_labels, verbose=2)
print(f'BaseNet-32 (Adam) Testing Accuracy: {test_acc_base_32_adam:.4f}')
print(f'BaseNet-32 (Adam) Testing Loss: {test_loss_base_32_adam:.4f}')


Epoch 1/10
782/782 [==============================] - 21s 25ms/step - loss: 2.1688 - accuracy: 0.1746 - lr: 0.1000
Epoch 2/10
782/782 [==============================] - 19s 25ms/step - loss: 2.1445 - accuracy: 0.1791 - lr: 0.1000
Epoch 3/10
782/782 [==============================] - 19s 25ms/step - loss: 2.1418 - accuracy: 0.1690 - lr: 0.1000
Epoch 4/10
782/782 [==============================] - 19s 25ms/step - loss: 2.0912 - accuracy: 0.1835 - lr: 0.1000
Epoch 5/10
782/782 [==============================] - 19s 25ms/step - loss: 2.0887 - accuracy: 0.1853 - lr: 0.1000
Epoch 6/10
782/782 [==============================] - 20s 25ms/step - loss: 2.0805 - accuracy: 0.1910 - lr: 0.1000
Epoch 7/10
782/782 [==============================] - 19s 25ms/step - loss: 2.0565 - accuracy: 0.1919 - lr: 0.1000
Epoch 8/10
782/782 [==============================] - 19s 25ms/step - loss: 2.0677 - accuracy: 0.1962 - lr: 0.1000
Epoch 9/10
782/782 [==============================] - 19s 25ms/step - loss: 2.06

In [147]:
# Initialize the legacy Adam optimizer
adam = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)


# Train and evaluate ResNet-32 with the legacy Adam optimizer
model_resnet_32_adam = create_resnet(num_layers=32)
model_resnet_32_adam.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model_resnet_32_adam.fit(train_images, train_labels, batch_size=64, epochs=10, callbacks=[LearningRateScheduler(lr_schedule)])
scores_resnet_32_adam = model_resnet_32_adam.evaluate(train_images, train_labels, verbose=0)
print("ResNet-32 (Adam) Training Accuracy:", scores_resnet_32_adam[1])
print("ResNet-32 (Adam) Training Loss:", scores_resnet_32_adam[0])

test_loss_resnet_32_adam, test_acc_resnet_32_adam = model_resnet_32_adam.evaluate(test_images, test_labels, verbose=2)
print(f'ResNet-32 (Adam) Testing Accuracy: {test_acc_resnet_32_adam:.4f}')
print(f'ResNet-32 (Adam) Testing Loss: {test_loss_resnet_32_adam:.4f}')



Epoch 1/10
782/782 [==============================] - 24s 28ms/step - loss: 2.0801 - accuracy: 0.2012 - lr: 0.1000
Epoch 2/10
782/782 [==============================] - 22s 28ms/step - loss: 1.7365 - accuracy: 0.3361 - lr: 0.1000
Epoch 3/10
782/782 [==============================] - 22s 28ms/step - loss: 1.6310 - accuracy: 0.3879 - lr: 0.1000
Epoch 4/10
782/782 [==============================] - 22s 28ms/step - loss: 1.9037 - accuracy: 0.3020 - lr: 0.1000
Epoch 5/10
782/782 [==============================] - 22s 28ms/step - loss: 1.7161 - accuracy: 0.3277 - lr: 0.1000
Epoch 6/10
782/782 [==============================] - 22s 28ms/step - loss: 1.5743 - accuracy: 0.3959 - lr: 0.1000
Epoch 7/10
782/782 [==============================] - 22s 28ms/step - loss: 1.4757 - accuracy: 0.4424 - lr: 0.1000
Epoch 8/10
782/782 [==============================] - 22s 28ms/step - loss: 1.4037 - accuracy: 0.4772 - lr: 0.1000
Epoch 9/10
782/782 [==============================] - 22s 28ms/step - loss: 1.33